Minando datos de 8ch.net con python
====================

[![Anaconda-Server Badge](https://anaconda.org/bc-privsec-devel/chanscrape/badges/license.svg)](https://anaconda.org/bc-privsec-devel/chanscrape)

**Esta es la libreta #2 de 2 en esta serie. **

El codigo fuente, asi como instrucciones para instalar y ejecutar esta libreta por su propia cuenta, estan disponibles en su [repositorio oficial de Github](https://github.com/BC-PRIVSEC/ChanScrape). Los autores y colaboradores en el desarrollo de este software hemos decidido **dedicar este trabajo al Dominio Publico** renunciando a cualquier derecho de autor _en detrimento propio y de nuestros sucesores_; y para beneficio de el publico general. Consulte los [detalles legales en el Repositorio oficial](https://github.com/BC-PRIVSEC/ChanScrape/LICENCIA-ES).

Queremos demostrar de esta manera que **no tenemos interes alguno en lucrar** con las victimas, por el contrario el objetivo es proporcionarles herramientas informaticas y conocimiento en mejores practicas para **defender su seguridad, integridad e identidad en Linea.** 

#### Use esta libreta y la herramienta que representa para los fines legales que a usted convenga.
------------

###  Recabando todos los datos que sea posible

Ya hemos visto que la libreria `py8chan` y el API de la misma pagina resultan en una poderosa combinacion. En esta entrega, haremos unas sencillas modificaciones al programa que escribimos en la libreta anteror para lograr lo siguiente:

- Recabar correos electronicos de todos los mensajes
- Adiconalmente recabar todas las estadisticas que nos sea posible del api
- Organizar y grabar esta informacion de manera eficiente 
- Hacer de esta operacion una tarea automatizada y rutinaria de manera que podamos minar la informacion constantemente y sin ser penalizados por el sitio. 
- Experimentar con los datos para ver que conclusiones respecto a la comunidad monitoreada podemos obtener, asi como sus habitos y de ser posible, perfilar usuarios individuales

Definiremos una lista de los sitios que conocemos que operan bajo el mimo esquema, practicamente se les podria considerar una sola comunidad, por varias razones que estan mas alla del alcance tecnico de esta libreta. definiremos entonces la siguiente estructura de datos como la lista maestra de tableros a monitorear:

In [11]:
import py8chan #iniciamos el programa conel encabezado importando las librerias que ocuparemos 
import json
import re

from datetime import datetime
from bs4 import BeautifulSoup

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

Definimos de una buena vez las funciones auxiliares adicionales, seran las mismas que las utilizadas en la libreta 1


In [12]:
def alt_unique(input_list):
    ''' implementacion eficiente de elementos uniicos usando
    conversion intermedia a set() que no permite duplicados. de
    esta manera evitamos caminar por cada elemento de la lista en 
    un bucle convencional'''
    output = set(input_list)
    return list(output)

Definiremos Nuestro inventario de datos maestro como una listas nativa de _Phyton_ para aprovechar las caracteristicas y prestaciones que el lenguaje define para los objetos de este tipo.

En particular es util que se comportan como _Generadores_ e _Iteradores_ lo que nos ahorrara muchas lineas de codigo. Cada uno de los elementos de esta lista sern los nombres de los tableros de interes en dicho sitio. 


In [13]:
Inventario_Maestro =['ensenada', 'tijuana', 'mexicali', 
                     'mexicaligirls', 'mexicalilocal', 
                     'mexicali2', 'drive646']

timestamp = datetime.isoformat( datetime.now()) #Hora exacta como sello para identificar la corrida
patron_email = re.compile(r'[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,4}')
emails = []
mensajes = 0
total_conversaciones=0
total_mensajes = 0


### Enlace al API e instancias de objetos de datos

Aplicaremos la propiedad generadora de la lista:

In [14]:
for nombre_de_tablero in Inventario_Maestro:
    tablero = py8chan.Board(nombre_de_tablero) 
    conversaciones = tablero.get_all_threads() 
    total_conversaciones += len(conversaciones)

    print('Procesando el tablero', nombre_de_tablero, '...')
    print('Hay ',len(conversaciones), 'conversaciones en', nombre_de_tablero)

    mensajes_del_tablero = 0

    for conversacion in conversaciones:
        mensajes += len(conversacion.all_posts)     
        mensajes_del_tablero += len(conversacion.all_posts)
        for msg in conversacion.all_posts:          
            em = patron_email.findall(msg.comment)  
            if em:                                  
                for m in em:                        
                    emails.append(m)
                    
    print('Hay ', mensajes_del_tablero, 'mensajes en total para el tablero', nombre_de_tablero)
    print(' ')
                    
unique_emails = alt_unique(emails)              
unique_emails.sort()                


Procesando el tablero ensenada ...
Hay  375 conversaciones en ensenada
Hay  3174 mensajes en total para el tablero ensenada
 
Procesando el tablero tijuana ...
Hay  375 conversaciones en tijuana
Hay  2170 mensajes en total para el tablero tijuana
 
Procesando el tablero mexicali ...
Hay  375 conversaciones en mexicali
Hay  2472 mensajes en total para el tablero mexicali
 
Procesando el tablero mexicaligirls ...
Hay  375 conversaciones en mexicaligirls
Hay  3953 mensajes en total para el tablero mexicaligirls
 
Procesando el tablero mexicalilocal ...
Hay  124 conversaciones en mexicalilocal
Hay  416 mensajes en total para el tablero mexicalilocal
 
Procesando el tablero mexicali2 ...
Hay  83 conversaciones en mexicali2
Hay  664 mensajes en total para el tablero mexicali2
 
Procesando el tablero drive646 ...
Hay  6 conversaciones en drive646
Hay  50 mensajes en total para el tablero drive646
 


In [15]:
print('Se han procesado', len(Inventario_Maestro) , 'tableros')
print('Hemos capturado ', total_conversaciones, 'conversaciones')
print('Contienen ', mensajes, 'mensajes en total')
print('Se extrajeron ', len(unique_emails), 'emails unicos de', len(emails), 'instancias')
print('eso es cada', round(mensajes/len(emails),2), 'mensajes alguien pide envios y proporciona su direccion')

Se han procesado 7 tableros
Hemos capturado  1713 conversaciones
Contienen  12899 mensajes en total
Se extrajeron  956 emails unicos de 1759 instancias
eso es cada 7.33 mensajes alguien pide envios y proporciona su direccion


In [17]:

filename = "data/solo-email-todos-los-boards/" + timestamp.split(".")[0] + ".json"

with open(filename,'w+') as storage:
    storage.write(json.dumps(unique_emails))

print("Si no hay ningun error,los corrreos se han grabado en el archivo ", filename )
        

25396

Si no hay ningun error,los corrreos se han grabado en el archivo  data/solo-email-todos-los-boards/2018-02-08T22:42:29.json


In [33]:
%%bash
ls -lR data


data:
total 4
drwxrwxrwx 2 nobody nogroup 4096 Feb  8 23:28 solo-email-todos-los-boards

data/solo-email-todos-los-boards:
total 84
-rw-rw-rw- 1 nobody nogroup 25372 Feb  8 16:56 2018-02-08T16:34:47.json
-rw-rw-rw- 1 nobody nogroup 25372 Feb  8 20:23 2018-02-08T19:09:59.json
-rw-rw-rw- 1 nobody nogroup 25396 Feb  8 22:45 2018-02-08T22:42:29.json
